In [ ]:
import ee, geemap, os
import numpy as np
from subprocess import call
from datetime import datetime
from osgeo import gdal
from matplotlib import pyplot as plt

import numpy as np
import geopandas as gpd
import pymannkendall as mk
import pandas as pd
import time, os, math, statistics, shutil
import xarray as xr
import warnings
warnings.filterwarnings('ignore')
ee.Authenticate()
ee.Initialize()
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
Map = geemap.Map(center=(40, -100), zoom=1,height = 600,width = 800)
Map

In [ ]:
'''MK trend analysis'''
# 上升趋势为1，下降为-1，不变为0
trend_list = []
for i in range(0,len(df['FID'])):
    data = df.iloc[i,2:-2].tolist()
    data = list(filter(lambda x: x != 0, data))
    if len(data) <= 10:
        print(0)
    try:
        res = mk.original_test(data,alpha = 0.1)    
        print(f'trend:{res.trend}','p_value:{:.2f}'.format(res.p),'slope:{:.2f}'.format(res.slope),sep = ',')
        print(round(res.slope,2))
        if res.trend == 'no trend':
            trend_list.append(0)
            # print(0)
        if res.trend == 'increasing':
            trend_list.append(1)
            # print(1)
        if res.trend == 'decreasing':
            trend_list.append(-1)
            # print(-1)
    except:
        # print('none')
        trend_list.append(0)

In [ ]:
'''Statistics test'''
from scipy.stats import shapiro, kstest, weibull_min, ttest_ind, mannwhitneyu
data = group_b

_, p_value = shapiro(data)
print(f"Shapiro-Wilk  p value: {p_value:.4f}")
if p_value < 0.1:
    print("0")
else:
    print("1")


sample1 = group_a
sample2 = group_b

# Welch's two sample t-test
t_statistic, p_value = ttest_ind(sample1, sample2, equal_var=False)

print("t-statistic:", t_statistic)
print("p-value:", p_value) 


group_a = df['median'].dropna()
group_b = df['median.1'].dropna()

# 进行 Mann-Whitney U 检验
stat, p_value = mannwhitneyu(group_a, group_b)

# 输出结果
print(f"Mann-Whitney U : {stat:.2f}")
print(f"p : {p_value:.4f}")

alpha = 0.1  
if p_value < alpha:
    print("0")
else:
    print("1")

In [ ]:
'''Fig. 1b'''
mpl.rcParams['font.size'] = 13

df = pd.read_excel(r"C:\Users\DELL\Desktop\新SSC数据.xlsx",sheet_name='流域各断面中均值')
    
columns =  ['yukon', 'mackenzie','kolyma',  'pechora', 'ob','lena', 'khatnaga',  'yenisey',
         'nelson', 'thelon','severnaya',]

data = [df[column].dropna() for column in columns]

fig, ax = plt.subplots(1,1,figsize=(10,3.6),dpi = 300)
# box_colors = ['#a50026','#d73027','#f46d43','#fdae61','#fee090','#ffffbf','#e0f3f8','#abd9e9','#74add1','#4575b4','#313695']
# box_colors = ['#E3E4E8'] * 11
box_colors = ["#d9ed92", "#b5e48c", "#99d98c", "#76c893", "#52b69a", "#34a0a4", "#168aad", "#1a759f", "#1e6091", "#184e77",'grey']
ax.spines['top'].set_visible(False)  
ax.spines['right'].set_visible(False)  


boxplot = plt.boxplot(data, labels=[column.capitalize() for column in columns], vert=True,showfliers=False,patch_artist=True)
for box, color in zip(boxplot['boxes'], box_colors):
    box.set(facecolor=color, alpha = 0.6)
plt.ylim(0, 400)

# fig.text(0.8, 0.8, '2004−2023 vs. 1984−2003', color='k', ha='center', va='bottom', fontstyle='italic', fontweight='normal')
ax.set_ylabel('SSC (mg/L)')
for median in boxplot['medians']:
    median.set(color='k')
plt.tight_layout()

In [ ]:
'''Fig. 2b'''
from matplotlib.gridspec import GridSpec
mpl.rcParams['font.size'] = 13

fig = plt.figure(figsize=(11, 3.1),dpi = 300)

gs = GridSpec(1, 2, width_ratios=[8, 2])
ax1 = plt.subplot(gs[0])
ax1.set(xlim=(1983.5, 2023.5), ylim = (30,110))
ax1.yaxis.set_major_locator(MultipleLocator(20))

colors = ['#d6604d','#0571b0']  #   '#3E7545','#F2BF25'

legend = []
data3 = pd.read_excel(r"C:\Users\DELL\Desktop\新SSC数据.xlsx",sheet_name='ssc_median')
American = data3['American']
Eurasia = data3['Eurasia'][16:]
year_columns = data3['year']
for data, color, label in zip([American, Eurasia],colors,['American','Eurasia']):
    if label == 'Eurasia':
        year_columns = year_columns = data3['year'][16:]
    else:
        year_columns = year_columns = data3['year']
    regression_line = linear_regression(year_columns, data)
    confidence_interval = confidence(year_columns, data)
    line = ax1.plot(year_columns, regression_line, color=color, lw = 1.5)
    ax1.fill_between(year_columns, regression_line - confidence_interval, regression_line + confidence_interval, color=color, alpha=0.15, edgecolor='none', linestyle='-', lw=1)
    ax1.plot(year_columns, data, color=color,lw = 2.3,label = label, ls = '--')
    legend.extend(line)
ax1.legend(legend, ['North America','Eurasia'], loc='upper right', 
           markerfirst=True, ncol=1, frameon=False,bbox_to_anchor=(1.05, 1.05))

year_columns = data3['year']

ax2 = plt.subplot(gs[1])

ax2.yaxis.tick_right()
ax2.set(ylim = (-0.5,5.8))
ax2.yaxis.set_label_position('right')
ax2.set_ylabel('Change rate of SSC (mg/l/yr)', rotation=270)
ax2.yaxis.set_label_coords(1.25, 0.5)  # 调整标签位置
ax2.yaxis.set_major_locator(MultipleLocator(2))

# ax2.grid(True, linestyle='-', which='both', color='white', alpha=0.8, lw=1.5)
data = pd.read_excel(r"C:\Users\DELL\Desktop\新SSC数据.xlsx",sheet_name='各断面SSC的slope')

data1 = []
for column in  ['kolyma', 'lena', 'yenisey', 'khatnaga', 'ob']:
    column_data = data[column].dropna().tolist()
    data1.extend(column_data)
data2 = []
for column in  ['nelson','mackenzie','yukon','thelon']:
    column_data = data[column].dropna().tolist()
    data2.extend(column_data)
    
boxplot = ax2.boxplot([data2,data1],widths = 0.4,patch_artist=True,
                flierprops={'markerfacecolor': 'grey', 'markeredgecolor': 'none', 'marker': 'o','alpha':0.7},
                medianprops={'color': 'white'},
                boxprops={'color': 'gray'})


for box, color in zip(boxplot['boxes'], colors):
    box.set(facecolor=color, alpha = 0.9)
means = [np.mean(data2), np.mean(data1)]

# 绘制平均值
for i, mean in enumerate(means, start=1):
    ax2.plot(i, mean, 'v', markersize=8, c = 'white')
ax1.set_ylabel('Median SSC (mg/L)')
ax1.spines['top'].set_visible(False) 
ax2.spines['top'].set_visible(False) 
ax1.spines['right'].set_visible(False) 
ax2.spines['left'].set_visible(False) 

plt.subplots_adjust(wspace=0.07)